# LangGraph To-Do App with Subgraph Reminder

This notebook refactors the original Python script into an interactive walkthrough that showcases how to build a to-do list agent with a reminder-generating subgraph using LangGraph.



#### Subgraph Visualization

<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace">   +-----------+     
   | __start__ |     
   +-----------+     
          *          
          *          
          *          
  +------------+     
  | prioritize |     
  +------------+     
          *          
          *          
          *          
+-----------------+  
| format_reminder |  
+-----------------+  
          *          
          *          
          *          
    +---------+      
    | __end__ |      
    +---------+      
</pre>

#### Main Graph Visualization

<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace">    +-----------+      
    | __start__ |      
    +-----------+      
          *            
          *            
          *            
    +----------+       
    | add_todo |       
    +----------+       
          *            
          *            
          *            
+-------------------+  
| reminder_subgraph |  
+-------------------+  
          *            
          *            
          *            
  +---------------+    
  | display_todos |    
  +---------------+    
          *            
          *            
          *            
     +---------+       
     | __end__ |       
     +---------+       
</pre>

## 1. Imports and Setup
Import required typing helpers, LangChain / LangGraph utilities, and Rich for styled logging.


In [17]:
from typing import List, TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, END, START
from rich import print

## 2. Define Shared State
We describe the schema for data that flows through the graph using a `TypedDict`. The `todos` list accumulates across graph invocations via `operator.add`.

In [18]:
class TodoState(TypedDict):
    """
    Represents the application's state, including the list of tasks and reminders.
    """

    # Use operator.add to accumulate items into the list across graph runs.
    todos: Annotated[List[str], operator.add]
    reminder: str
    new_todo: str

## 3. Main Graph Nodes
These nodes handle task ingestion and displaying final results.

In [19]:
# Define a node to add a new task to the to-do list.
def add_todo(state: TodoState):
    """
    Appends a new task from the 'new_todo' field to the 'todos' list.
    """
    print('--- ADDING TODO ---')
    new_task = state.get('new_todo', '').strip()

    # Handle cases where the input task is empty.
    if not new_task:
        print('No new task provided.')
        return {'todos': []}  # Return an empty list to signify no change.

    print(f'Adding task: {new_task}')
    # Return the new task in a list to be appended to the state.
    return {'todos': [new_task]}


# Define a node to print the final state of the to-do list and reminder.
def display_final_state(state: TodoState):
    """
    Displays the final list of tasks and the generated reminder to the console.
    """
    print('\n--- FINAL TO-DO LIST & REMINDER ---')
    print(f'Tasks: {state["todos"]}')
    print(f'Reminder: {state["reminder"]}')
    print('------------------------------------')
    return {}  # Return an empty dictionary as this node doesn't modify state.


## 4. Subgraph Nodes
The subgraph uses an LLM to prioritize tasks and then formats the reminder message.

In [20]:

# Define a node that uses an LLM to prioritize the current list of tasks.
def prioritize_tasks_llm(state: TodoState):
    """
    Invokes an LLM to generate a prioritized list from the current 'todos'.
    """
    print('\n--- SUBGRAPH: PRIORITIZING TASKS ---')
    llm = ChatOllama(model='llama3.2')  # Initialize the language model.

    # If there are no tasks, return a default message.
    if not state['todos']:
        return {'reminder': 'No tasks to prioritize.'}

    # Format the list of tasks into a string for the LLM prompt.
    task_list_str = '\n'.join(f'- {task}' for task in state['todos'])

    # Construct the prompt with system and human messages.
    prompt = [
        SystemMessage(content='You are a helpful assistant. Your task is to prioritize a list of to-do items.'),
        HumanMessage(
            content=f'Please prioritize the following tasks and return them as a numbered list, with the most important task first:\n\n{task_list_str}'
        ),
    ]

    response = llm.invoke(prompt)
    print('LLM generated prioritized list.')
    # Update the 'reminder' field with the raw LLM output.
    return {'reminder': response.content}


# Define a node to format the LLM's output into a user-friendly message.
def format_reminder_text(state: TodoState):
    """
    Wraps the prioritized task list in a friendly reminder message.
    """
    print('--- SUBGRAPH: FORMATTING REMINDER ---')
    prioritized_tasks = state.get('reminder', 'No tasks were prioritized.')
    formatted_reminder = f'Here is your prioritized reminder for today:\n{prioritized_tasks}'
    # Overwrite the 'reminder' field with the final formatted string.
    return {'reminder': formatted_reminder}


## 5. Build Graphs
Create the reminder subgraph and integrate it into the main application graph.
### Building Subgraph

In [26]:

# Create the state graph for the reminder-generation subgraph.
reminder_subgraph = StateGraph(TodoState)
# Add nodes for prioritizing tasks and formatting the output.
reminder_subgraph.add_node('prioritize', prioritize_tasks_llm)
reminder_subgraph.add_node('format_reminder', format_reminder_text)

# Define the data flow within the subgraph.
reminder_subgraph.add_edge(START, 'prioritize')
reminder_subgraph.add_edge('prioritize', 'format_reminder')
reminder_subgraph.add_edge('format_reminder', END)

# Compile the subgraph into a runnable application.
reminder_app = reminder_subgraph.compile()
print(reminder_app.get_graph().draw_ascii())


+-----------+     
   | __start__ |     
   +-----------+     
          *          
          *          
          *          
  +------------+     
  | prioritize |     
  +------------+     
          *          
          *          
          *          
+-----------------+  
| format_reminder |  
+-----------------+  
          *          
          *          
          *          
    +---------+      
    | __end__ |      
    +---------+

### Building Main Graph

In [27]:

# Create the main application graph.
main_graph = StateGraph(TodoState)

# Add nodes for the main workflow.
main_graph.add_node('add_todo', add_todo)
# Add the compiled subgraph as a single, callable node.
main_graph.add_node('reminder_subgraph', reminder_app)
main_graph.add_node('display_todos', display_final_state)

# Define the data flow for the main graph.
main_graph.add_edge(START, 'add_todo')
main_graph.add_edge('add_todo', 'reminder_subgraph')
main_graph.add_edge('reminder_subgraph', 'display_todos')
main_graph.add_edge('display_todos', END)

# Compile the complete main graph into the final application.
app = main_graph.compile()
print(app.get_graph().draw_ascii())

+-----------+      
    | __start__ |      
    +-----------+      
          *            
          *            
          *            
    +----------+       
    | add_todo |       
    +----------+       
          *            
          *            
          *            
+-------------------+  
| reminder_subgraph |  
+-------------------+  
          *            
          *            
          *            
  +---------------+    
  | display_todos |    
  +---------------+    
          *            
          *            
          *            
     +---------+       
     | __end__ |       
     +---------+

## 6. Visualize Graph Structure
Optionally render the LangGraph diagram (requires graphviz support).

Subgraph structure:

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

## 7. Run the Application
Simulate consecutive invocations of the graph to add tasks and generate a reminder.

In [24]:
# Initialize an empty state for the first run.
current_state = {'todos': [], 'reminder': ''}

# Execute the graph sequentially to add multiple tasks and update the state.
# First invocation adds 'Finish LangGraph tutorial video'.
inputs1 = {'new_todo': 'Finish LangGraph tutorial video'}
current_state = app.invoke(
    inputs1, config={'recursion_limit': 100}, state=current_state
)  # Pass previous state to accumulate tasks.

# Second invocation adds 'Buy groceries'.
inputs2 = {'new_todo': 'Buy groceries'}
current_state = app.invoke(inputs2, config={'recursion_limit': 100}, state=current_state)  # Pass previous state.

# Third invocation adds 'Schedule dentist appointment' and gets the final result.
inputs3 = {'new_todo': 'Schedule dentist appointment'}
final_state = app.invoke(inputs3, config={'recursion_limit': 100}, state=current_state)  # Pass previous state.

--- ADDING TODO ---

Adding task: Finish LangGraph tutorial video

--- SUBGRAPH: PRIORITIZING TASKS ---

LLM generated prioritized list.

--- SUBGRAPH: FORMATTING REMINDER ---

...

--- SUBGRAPH: FORMATTING REMINDER ---

--- FINAL TO-DO LIST & REMINDER ---

Tasks: ['Schedule dentist appointment', 'Schedule dentist appointment']

Reminder: Here is your prioritized reminder for today:
Here's the prioritized list of tasks:

1. **Schedule dentist appointment**
2. 
3. (Note: There is no other information provided about the remaining tasks. If you provide more context or details,
I can help prioritize them as well.)

------------------------------------